<H1>Table of Contents</H1>
<ul>
<li>Scrape Neighborhoods
<li>Scrape Coordinates
<li>Analyze Neighborhoods
<li>Cluster Neighborhoods
<li>Observations

<H1>Scrape Neighborhoods

In [1]:
# Import required libraries
import requests
import pandas as pd
import numpy as np
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [2]:
# Load website source code and retain only code containing table data
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
website_table = website_url.split('<tbody')
website_table = website_table[1].split('</tbody>')
# Split into rows
website_table = website_table[0].split("<tr>")
website_table[0:10]

['>',
 '\n<th>Postcode</th>\n<th>Borough</th>\n<th>Neighbourhood\n</th></tr>\n',
 '\n<td>M1A</td>\n<td>Not assigned</td>\n<td>Not assigned\n</td></tr>\n',
 '\n<td>M2A</td>\n<td>Not assigned</td>\n<td>Not assigned\n</td></tr>\n',
 '\n<td>M3A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>\n</td></tr>\n',
 '\n<td>M4A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>\n</td></tr>\n',
 '\n<td>M5A</td>\n<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>\n<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>\n</td></tr>\n',
 '\n<td>M5A</td>\n<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>\n<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>\n</td></tr>\n',
 '\n<td>M6A

In [3]:
# Split rows into columns
splitStr = "</td>\\n<td>"
website_list = []
for row in website_table:
    try:
        if row[5] == "M":
            row_df = row.split("<td>")
            website_list.append(row_df)
    except:
        continue

In [4]:
# Place cells into dataframe, clear strings from HTML tags, drop NaN cells
website_df = pd.DataFrame(website_list, columns=['Empty','PostalCode','Borough','Neighborhood'])
website_df.drop('Empty', axis=1, inplace=True)
website_df['PostalCode'].replace('</td>\\n', '', regex=True, inplace=True)
website_df['Borough'].replace('</td>\\n', '', regex=True, inplace=True)
website_df['Borough'].replace('<a.*?>', '', regex=True, inplace=True)
website_df['Borough'].replace('</a>', '', regex=True, inplace=True)
website_df['Neighborhood'].replace('\n</td></tr>\n', '', regex=True, inplace=True)
website_df['Neighborhood'].replace('<a.*?>', '', regex=True, inplace=True)
website_df['Neighborhood'].replace('</a>', '', regex=True, inplace=True)
website_df.replace('Not assigned', np.nan, regex=True, inplace=True)
website_df.dropna(subset=['Borough'], inplace=True)
website_df.reset_index(drop=True, inplace=True)
website_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [5]:
# Create Neighborhood column with same PostalCode
website_df2 = website_df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(map(str, x)))
website_df2 = website_df2.replace('nan', np.nan)
website_df2 = website_df2.to_frame()
website_df2['PostalCode'] = website_df2.index
website_df2.head()

,Neighborhood,PostalCode
PostalCode,,
M1B,"Rouge, Malvern",M1B
M1C,"Highland Creek, Rouge Hill, Port Union",M1C
M1E,"Guildwood, Morningside, West Hill",M1E
M1G,Woburn,M1G
M1H,Cedarbrae,M1H


In [6]:
# Drop duplicate PostalCodes/Boroughs and join with Neighborhood Series
website_df3 = website_df.drop_duplicates(subset=['PostalCode'], keep='first', inplace=False)
website_df3.drop(columns=['Neighborhood'], inplace=True)
website_df4 = pd.merge(website_df3, website_df2, on="PostalCode")
website_df4.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,NaN


In [7]:
# Fill NaN Neighborhood cells with Borough
website_df4['Neighborhood'].fillna(website_df4['Borough'], inplace=True)
website_df4.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [8]:
# Display dataframe size
website_df4.shape

(103, 3)

---

<H1>Scrape Coordinates

In [9]:
# Read Neighborhood coordinates from csv into dataframe, as Geocoder is unreliable
coords_url = pd.read_csv('https://cocl.us/Geospatial_data')
coords_url.rename(index=str, columns={"Postal Code": "PostalCode"},inplace=True)
coords_url.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Join Coordinated and Neighborhood dataframes based on PostalCode value
website_df5 = pd.merge(website_df4, coords_url, on="PostalCode")
website_df5.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [11]:
# Display dataframe size
website_df5.shape

(103, 5)

---

<H1>Analyze Neighborhoods

In [12]:
latitude = 43.761539
longitude = -79.411079
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.761539, -79.411079.


In [13]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = website_df5

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
CLIENT_ID = '3DO3N550RX4XMAL3WG2TOTCILJBUYE2DDOMIKA410AP4KRKC' # your Foursquare ID
CLIENT_SECRET = 'YNDFTPZXCG3NGCT5MUGRCEMPKBPZMPZPY2LYWOSQ4YLOEBOB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100
radius = 500

Your credentails:
CLIENT_ID: 3DO3N550RX4XMAL3WG2TOTCILJBUYE2DDOMIKA410AP4KRKC
CLIENT_SECRET:YNDFTPZXCG3NGCT5MUGRCEMPKBPZMPZPY2LYWOSQ4YLOEBOB


In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(2230, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,55,55,55,55,55,55


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_onehot.shape

(2230, 268)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.010000
1,Agincourt,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Alderwood, Long Branch",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.0,0.000000
6,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [24]:
toronto_grouped.shape

(101, 268)

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1          Lounge  0.25
2  Clothing Store  0.25
3    Skating Rink  0.25
4  Massage Studio  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2             Massage Studio   0.0
3  Middle Eastern Restaurant   0.0
4         Mexican Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.18
1  Fried Chicken Joint  0.09
2          Coffee Shop  0.09
3             Pharmacy  0.09
4          Pizza Place  0.09


----Alderwood, Long Branch----
                venue  fr

                       venue  freq
0                Coffee Shop  0.33
1          Convenience Store  0.33
2                       Park  0.33
3  Middle Eastern Restaurant  0.00
4         Mexican Restaurant  0.00


----Emery, Humberlea----
                venue  freq
0      Baseball Field   1.0
1         Yoga Studio   0.0
2   Martial Arts Dojo   0.0
3  Mexican Restaurant   0.0
4       Metro Station   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.12
1  Fast Food Restaurant  0.07
2           Coffee Shop  0.06
3      Asian Restaurant  0.04
4         Women's Store  0.03


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.12
1         Café  0.08
2        Hotel  0.06
3   Restaurant  0.05
4   Steakhouse  0.04


----Flemingdon Park, Don Mills South----
              venue  freq
0        Beer Store  0.10
1               Gym  0.10
2       Coffee Shop  0.10
3  Asian Restaurant  0.10
4     Grocery Store  0.05




                venue  freq
0   Convenience Store  0.33
1                 Spa  0.33
2          Playground  0.33
3   Martial Arts Dojo  0.00
4  Mexican Restaurant  0.00


----Silver Hills, York Mills----
                       venue  freq
0                  Cafeteria   1.0
1                Yoga Studio   0.0
2             Medical Center   0.0
3  Middle Eastern Restaurant   0.0
4         Mexican Restaurant   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.06
2      Restaurant  0.05
3           Hotel  0.04
4  Clothing Store  0.04


----Stn A PO Boxes 25 The Esplanade----
          venue  freq
0   Coffee Shop  0.11
1          Café  0.04
2         Hotel  0.03
3  Cocktail Bar  0.03
4      Beer Bar  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2               Bakery  0.05
3   Italian Restaurant  0.05
4  American Restaurant  0.05


----The Annex, North Midtown, Yorkville---

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Bar,Cosmetics Shop,Hotel,Restaurant,Gym
1,Agincourt,Breakfast Spot,Clothing Store,Skating Rink,Lounge,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Sandwich Place,Discount Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Beer Store
4,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Coffee Shop,Skating Rink,Gym,Pub,Bank,Sandwich Place,Pharmacy,Pool
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fried Chicken Joint,Sandwich Place,Diner,Bridal Shop,Restaurant,Bank,Deli / Bodega,Ice Cream Shop,Supermarket
6,Bayview Village,Bank,Café,Chinese Restaurant,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Sushi Restaurant,Italian Restaurant,Fast Food Restaurant,Comfort Food Restaurant,Pharmacy,Pizza Place,Café,Butcher,Pub
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Cheese Shop,Farmers Market,Seafood Restaurant,Steakhouse,Café,Beer Bar
9,"Birch Cliff, Cliffside West",Skating Rink,College Stadium,General Entertainment,Café,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


---

<H1>Cluster Neighborhoods

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 4, 0, 0, 3, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2,
       0, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 2], dtype=int32)

In [29]:
toronto_merged = neighborhoods

print(toronto_merged.shape)
print(kmeans.labels_.shape)

# Due to index mismatch, the cluster labels are added to the Grouped dataframe which is joined to the Merged dataframe

toronto_grouped_clustering['Cluster Labels'] = kmeans.labels_
# add clustering labels
toronto_merged['Cluster Labels'] = toronto_grouped_clustering['Cluster Labels']
# Print and Delete NaN neighborhoods with no venues
print(toronto_merged[toronto_merged['Cluster Labels'].isnull()])
toronto_merged.dropna(inplace=True)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head() # check the last columns!


(103, 5)
(101,)
    PostalCode    Borough                                       Neighborhood  \
101        M8Y  Etobicoke  Humber Bay, King's Mill Park, Kingsway Park So...   
102        M8Z  Etobicoke  Kingsway Park South West, Mimico NW, The Queen...   

      Latitude  Longitude  Cluster Labels  
101  43.636258 -79.498509             NaN  
102  43.628841 -79.520999             NaN  


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Fast Food Restaurant,Food & Drink Shop,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2.0,Coffee Shop,Park,Café,Bakery,Mexican Restaurant,Pub,Theater,Breakfast Spot,Yoga Studio,Italian Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Arts & Crafts Store,Event Space,Coffee Shop,Boutique,Shoe Store,Women's Store,Vietnamese Restaurant,Accessories Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Burger Joint,Sushi Restaurant,Japanese Restaurant,Gym,Diner,Spa,Smoothie Shop,College Auditorium,Seafood Restaurant


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Park,Fast Food Restaurant,Food & Drink Shop,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
1,North York,0.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
3,North York,0.0,Furniture / Home Store,Clothing Store,Arts & Crafts Store,Event Space,Coffee Shop,Boutique,Shoe Store,Women's Store,Vietnamese Restaurant,Accessories Store
4,Queen's Park,0.0,Coffee Shop,Burger Joint,Sushi Restaurant,Japanese Restaurant,Gym,Diner,Spa,Smoothie Shop,College Auditorium,Seafood Restaurant
5,Etobicoke,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Scarborough,0.0,Fast Food Restaurant,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
7,North York,0.0,Caribbean Restaurant,Gym / Fitness Center,Café,Japanese Restaurant,Pool,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
8,East York,0.0,Pizza Place,Fast Food Restaurant,Athletics & Sports,Pet Store,Rock Climbing Spot,Bank,Intersection,Pharmacy,Gastropub,Gym / Fitness Center
9,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Bar,Japanese Restaurant,Movie Theater,Middle Eastern Restaurant,Pizza Place,Italian Restaurant
10,North York,0.0,Pizza Place,Metro Station,Sushi Restaurant,Park,Pub,Japanese Restaurant,Dessert Shop,Curling Ice,Dance Studio,Deli / Bodega


In [32]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,Etobicoke,1.0,Park,Mobile Phone Shop,Pizza Place,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store


In [33]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2.0,Coffee Shop,Park,Café,Bakery,Mexican Restaurant,Pub,Theater,Breakfast Spot,Yoga Studio,Italian Restaurant
13,North York,2.0,Asian Restaurant,Gym,Coffee Shop,Beer Store,Grocery Store,General Entertainment,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Dim Sum Restaurant
16,York,2.0,Trail,Dog Run,Hockey Arena,Field,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
38,Scarborough,2.0,Coffee Shop,Discount Store,Department Store,Chinese Restaurant,Hobby Shop,Train Station,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner
40,North York,2.0,Park,Airport,Snack Place,Bus Stop,College Rec Center,College Gym,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
45,North York,2.0,Cafeteria,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Curling Ice
58,Scarborough,2.0,Skating Rink,College Stadium,General Entertainment,Café,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
62,Central Toronto,2.0,Garden,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Curling Ice
65,Scarborough,2.0,Indian Restaurant,Latin American Restaurant,Brewery,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
67,Central Toronto,2.0,Park,Hotel,Breakfast Spot,Sandwich Place,Burger Joint,Food & Drink Shop,Asian Restaurant,Clothing Store,Comic Shop,Dessert Shop


In [34]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,3.0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Fruit & Vegetable Store,Pizza Place,Lounge,Juice Bar
55,North York,3.0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Fast Food Restaurant,Comfort Food Restaurant,Pharmacy,Pizza Place,Café,Butcher,Pub


In [35]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,North York,4.0,Gym,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop


---
<H1>OBSERVATIONS:</H1>
<li>2 Neighborhoods returned no venues for clustering
<li>5 clusters: 2 with many venues, 3 with very few due to different ratio of common venue categories